In [9]:

import pandas as pd
import os

LIMIT = int(1e2)

BASE_DIR = 'airbus-ship-detection/'


TRAIN_PATH = BASE_DIR + 'train_v2/'
TEST_PATH = BASE_DIR + 'test_v2/'
MASKS_PATH = BASE_DIR + 'train_ship_segmentations_v2.csv'

train = os.listdir(TRAIN_PATH)
test = os.listdir(TEST_PATH)

print(f'Sz of train dataset : {len(train)} |{train[:LIMIT]}')
print(f'Sz of test dataset : {len(test)} |{test[:LIMIT]}')

Sz of train dataset : 192556 |['4a5b2532e.jpg', '34b973fb0.jpg', '133b13bda.jpg', '5ecf4344b.jpg', '96f178a96.jpg', 'b5b6373bb.jpg', 'a78a82b25.jpg', 'c50310bc5.jpg', '531054a2b.jpg', '4235d2d8f.jpg', 'b0f4d07ed.jpg', '0deadf346.jpg', '2a9e2ac00.jpg', 'fa5c0c8b7.jpg', '6098e0ec7.jpg', '46d5ace64.jpg', '4790fd62c.jpg', 'cb9e718ee.jpg', '7ae2572ee.jpg', '6076a2546.jpg', 'c5ab26d22.jpg', '271956629.jpg', '5d8c071c4.jpg', '567261abb.jpg', 'a5ca05522.jpg', '448a95362.jpg', 'bad7a9587.jpg', 'a0c72d497.jpg', '75f2305a4.jpg', 'f56fa13de.jpg', '3cdc5adb9.jpg', '944c752de.jpg', '0af151ad2.jpg', '66a93d57b.jpg', '64c3335d0.jpg', '98f33d745.jpg', '3cf656feb.jpg', '938c8d3d3.jpg', '7ef0386e6.jpg', '7e86c4e26.jpg', '2880e817b.jpg', '84eed2fa0.jpg', 'ffd6a9ac7.jpg', '10dfa91c8.jpg', 'ca18c4ece.jpg', 'bc5a522cb.jpg', '6e7ebfd3f.jpg', '83aadba9d.jpg', '804f0977d.jpg', '65ffb677f.jpg', '877c2026b.jpg', '673e1b210.jpg', 'dcbd287ff.jpg', '9de779fbe.jpg', 'c9b34541b.jpg', 'b17ae157f.jpg', '1d7ff6175.jpg', 

In [14]:
masks = pd.read_csv(MASKS_PATH)

ne = pd.notna(masks.EncodedPixels)
print(f'Photos with ships : {ne.sum()}')


Photos with ships : 81723
